
## El Protocolo de Contexto de Modelo (MCP)

El **Protocolo de Contexto de Modelo (MCP)** sigue una **arquitectura cliente-servidor**, en la que una aplicación cliente, como un IDE o un agente de chat, puede conectarse a un servidor local o remoto e intercambiar mensajes con él.

Aunque la mayoría de los usuarios ya está familiarizada con esta arquitectura por la forma simplificada en que los navegadores interactúan con los sitios web, un vistazo al diseño del **Protocolo de Servidor de Lenguaje (LSP)** puede ofrecer una buena introducción a la arquitectura de MCP.

En este notebook se presenta:

1. Una breve descripción de la arquitectura general de LSP y MCP, comparándolas y contrastándolas.
2. Los componentes principales del protocolo, sus funciones individuales y su interacción:

   * aplicación host
   * cliente MCP
   * servidor MCP
   * transportes MCP
   * herramientas de desarrollo para facilitar la implementación.

---

## De LSP a MCP

Durante la discusión sobre los orígenes de MCP, se destacó que este protocolo se **inspiró en el Protocolo de Servidor de Lenguaje (LSP)**, ampliamente utilizado en IDE y editores modernos.

Antes de LSP, era común tener IDE específicos para cada lenguaje, debido a que el soporte debía construirse por separado para cada caso: un ejemplo clásico del problema **MxN** (véase la Figura 1-4).

LSP resolvió este problema al establecer un protocolo mediante el cual las aplicaciones host podían comunicarse con servidores de lenguaje ejecutados en procesos separados, a través de una interfaz de comunicación estándar **JSON-RPC**.

Esto se implementa con:

* **código cliente** incluido en la aplicación host, encargado de gestionar las conexiones,
* **servidores de lenguaje**, que se ejecutan como procesos separados y que proporcionan capacidades adicionales.

De esta manera, los servidores de lenguaje entregan funcionalidades a la aplicación host en un formato y lenguaje universal.





<p align="center">
<img src="../images/lsp_architecture.png" width="600" height="150">
    Figura 1-1. Un ciclo hipotético de acción-retroalimentación en acción para un agente de codificación.
</p>


<p align="center">
<img src="../images/connection_lifecycle.png" width="600" height="150">

Figura 2-2. La arquitectura LSP consta de una aplicación host (el IDE) con un cliente y uno o más servidores de lenguaje. La aplicación host emite eventos al cliente, que pueden convertirse en una solicitud o notificación al servidor de lenguaje. El servidor de lenguaje responde a la solicitud con una respuesta, que el cliente retransmite a la aplicación host.
</p>



## Diferencias clave entre LSP y MCP

Si ya se tiene conocimiento previo de MCP, la similitud con la arquitectura LSP resulta evidente:

* **Aplicaciones host** que integran un cliente.
* **Clientes MCP**, que gestionan las conexiones.
* **Servidores MCP**, que amplían las capacidades de la aplicación host.
* **Capa de comunicación** basada en JSON-RPC.

Sin embargo, MCP presenta **diferencias significativas** respecto a LSP:

* **Enfoque**: LSP se centra en extender las capacidades de los IDE; MCP es independiente del tipo de aplicación host.
* **Requisito mínimo**: la aplicación host debe comportarse como agente, pero no es obligatorio que sea un sistema de edición de código.
* **Ubicación de servidores**:

  * En LSP, los servidores suelen ejecutarse como procesos locales que se inician con el IDE.
  * En MCP, los servidores pueden ser **locales o remotos**.

Por ejemplo, al añadir un servidor local a una aplicación como **Claude Desktop**, se incluye la definición con el comando para iniciar el servidor y los parámetros requeridos. La comunicación ocurre mediante una capa de transporte local (por lo general, **stdio**).

En el caso de los **servidores remotos**, estos se definen con una **URL** y suelen requerir autenticación mediante claves API o tokens OAuth. Aquí se utilizan transportes remotos como **Streamable HTTP**, que permiten transmitir mensajes JSON-RPC.

---

## Transporte HTTP Transmisible

El **HTTP Streamable** es la **capa de transporte predeterminada** para servidores MCP remotos y es compatible de manera nativa con los **SDK oficiales de MCP**.

Este transporte permite que los servidores MCP se ejecuten en procesos independientes y acepten múltiples conexiones externas. La comunicación se basa en métodos HTTP habituales (**GET** y **PUT**), con un indicador opcional para habilitar **streaming SSE** (Server-Sent Events) entre cliente y servidor remoto.

Más detalles sobre este transporte se encuentran en la **documentación oficial de MCP** y en el **Capítulo 5** de este libro.



## Servidores MCP: locales, remotos y de terceros

Los **servidores MCP** pueden ser alojados de distintas formas:

* Por un tercero en sus propios servidores web.
* Dentro de una organización, en su **red privada**.

Esta flexibilidad **potencia el uso de la IA agentica**, ya que las herramientas ahora pueden distribuirse con la misma facilidad con la que se consumen las **API REST** en aplicaciones web tradicionales.

**Figura 2-3.** Interacción entre aplicaciones host, clientes y servidores MCP.

> Si la figura parece similar a la Figura 2-1, es porque esencialmente lo es, y refleja la **profunda influencia de LSP** en el diseño de MCP.

En esta arquitectura:

* El **cliente MCP** se conecta a la **aplicación host** y actúa como un **punto de nexo** entre la aplicación (y su LLM) y el servidor MCP.
* La aplicación host puede llamar al cliente, que envía la solicitud al servidor y luego recibe la respuesta.
* Para procesos como **elicitaciones y muestreo** (Capítulo 3), el flujo de comunicación puede invertirse según la tarea.
* Además, **clientes y servidores pueden enviarse notificaciones** entre sí, que no requieren una respuesta inmediata de la otra parte.

Esta estructura permite **una comunicación flexible y bidireccional**, esencial para la integración de herramientas, prompts y datos dentro del ecosistema MCP.



<img src="../images/mcp_architecture.png" width="600" height="150">

Figura 2-3. La arquitectura MCP consta de una aplicación host con un cliente y uno o más servidores MCP (solo una instancia de cliente puede conectarse a un solo servidor). La aplicación host llama al cliente, quien puede convertirlas en una solicitud o notificación al servidor MCP. El servidor MCP responde a la solicitud con una respuesta, que el cliente retransmite a la aplicación host. Mediante elicitación y muestreo, el servidor MCP también puede enviar solicitudes al cliente, quien las gestionará con la aplicación o LLM y enviará una solicitud de vuelta al servidor.

## La Aplicación Host y el Cliente MCP

Comenzaremos con las **aplicaciones host** y el **cliente MCP**. Por supuesto, las aplicaciones host no forman técnicamente parte de la arquitectura MCP, pero son necesarias para usar MCP. Las aplicaciones host son aquellas que albergan un cliente MCP y lo utilizan para ampliar sus capacidades mediante la interacción con servidores MCP. No hay restricciones sobre qué tipo de aplicaciones pueden ser host; algunos tipos comunes son **IDEs**, aplicaciones de chat como **Claude Desktop**, frameworks para construir agentes y terminales con capacidades de IA como **Warp**.

El **cliente** forma parte de la aplicación host y se encarga de **gestionar las conexiones con los servidores MCP**, así como de **rutar las solicitudes del LLM** al servidor correspondiente y de vuelta al LLM. Cada instancia de cliente mantiene una **conexión 1:1 con un único servidor**, retransmitiendo mensajes entre la aplicación host y el servidor. Los clientes también pueden determinar qué funcionalidades del servidor estarán disponibles para la aplicación host y el LLM. Algunos clientes pueden permitir únicamente el uso de las herramientas del servidor, mientras que otros pueden permitir el uso de todos los bloques de construcción del servidor (herramientas, recursos y prompts).

### Cliente o Cliente

Algunas fuentes pueden referirse a cualquier aplicación que consuma el contexto proporcionado por clientes MCP como “clientes”. En ciertos contextos, esto es un uso válido. Sin embargo, dado que este libro se centra en el código, hemos separado el término en **dos conceptos distintos**: la **aplicación host** y el **cliente MCP**.

* **Aplicación host:** la aplicación que **alberga el código del cliente MCP** y que interactúa con un LLM para ofrecer funcionalidades a los usuarios. Fuera de este punto de interacción con el LLM, el resto del código de la aplicación puede ser completamente independiente del cliente MCP.
* **Cliente MCP:** el **código específico dentro de la aplicación host** que se conecta a los servidores MCP y se comunica con ellos.

Los clientes también proporcionan algunas funciones adicionales a los servidores, como **sampling, roots y elicitation**, que aumentan la funcionalidad de los servidores. 

**Sampling** permite que un servidor llame al modelo de lenguaje de la aplicación host, habilitando flujos de trabajo complejos dentro de las herramientas del servidor.

Por ejemplo, un servidor MCP con acceso a la base de datos de ventas de la organización podría generar un informe de los últimos seis meses de ventas: el servidor llama a la herramienta de generación de informes, esta recupera los datos en bruto y los envía junto con plantillas y un prompt al cliente MCP, que a su vez utiliza el LLM de la aplicación host para generar el informe. Esto permite que el servidor controle los prompts y recursos necesarios para su funcionalidad sin depender directamente de un modelo de lenguaje propio.

**Figura 2-3** muestra visualmente este flujo de trabajo.



<img src="../images/sampling_sequence_diagram.png" width="800" height="150">




Figure 2-4. This sequence diagram shows an example sampling workflow. In it, the user asks the host application for a sales report. The host’s LLM chooses to call the sales reporting tool, which triggers the client to send a tool execution request to the server with the tool name and parameters. The server runs the code, and in its response returns the data, a report template, and a prompt. The client then can forward everything to the host LLM to generate the report and then return the result to the user.



**Roots** permiten a los clientes marcar partes específicas del sistema de archivos a las que un servidor conectado tiene acceso. Esto puede ser útil en muchos flujos de trabajo, como restringir automáticamente el acceso de un servidor a un directorio de proyecto específico o permitir que los usuarios seleccionen archivos y carpetas concretos para ponerlos a disposición de la aplicación y, por lo tanto, de cualquier servidor conectado. Es una forma potente de proporcionar contexto del sistema de archivos a los servidores que lo necesitan; sin embargo, no es un método seguro para limitar el alcance de un servidor malicioso. Esto se debe a que la especificación de MCP solo indica que los servidores deberían respetar cualquier root proporcionado por el cliente, pero no lo exige. Probablemente esto sirve como una función de compatibilidad para servidores que no acceden al sistema de archivos, pero el hecho de que los roots puedan ser ignorados los convierte en un sustituto deficiente para un sistema de control de acceso adecuado.

**Elicitations** son un componente relativamente nuevo en el conjunto de funciones del cliente MCP. Su propósito es similar al del *sampling*: mientras que *sampling* permite que el servidor haga llamadas al modelo de lenguaje de la aplicación host, las elicitations permiten que el servidor se conecte a la aplicación host a través del cliente y solicite la entrada del usuario. Esto significa que los servidores, cuando están conectados a clientes que soportan esta función, pueden construir no solo flujos de trabajo complejos impulsados por LLM, sino también flujos de trabajo que incluyan retroalimentación humana, proporcionando aún más flexibilidad y funcionalidad lista para usar a las aplicaciones host, siempre que el cliente de la aplicación lo soporte.

El **cliente** existe para proporcionar una conexión entre la aplicación host y el servidor MCP. Gestiona la conexión, retransmite mensajes entre el modelo de lenguaje de la aplicación host y el servidor, y ofrece funciones que permiten ejecutar flujos de trabajo más complejos desde el propio servidor. El **servidor** es el otro componente principal con el que te encontrarás como desarrollador MCP. Normalmente pasarás la mayor parte del tiempo trabajando con servidores, ya sea instalándolos para uso personal, conectándolos a tus propias aplicaciones agenticas o construyéndolos para que otros los utilicen.


**El Transporte MCP**

Los transportes en MCP implementan la capa de comunicación del protocolo. En su nivel más básico, son responsables de enviar y recibir mensajes entre el cliente y el servidor. Los SDK de MCP incluyen dos transportes predeterminados: *stdio* para conexiones locales y *Streamable HTTP* para conexiones remotas. Aunque estos son los más usados, el protocolo permite la implementación de transportes personalizados que pueden cubrir casos de uso para los cuales los transportes predeterminados no son los más adecuados, siempre que el canal de comunicación subyacente permita el paso bidireccional de mensajes. Los transportes pueden ser sin estado (como el transporte *stdio*) u opcionalmente con estado (como el transporte *Streamable HTTP*). En los transportes con estado, el transporte se encarga de mantener una sesión entre el cliente y el servidor, pudiendo soportar funcionalidades como reanudar tras una interrupción de red o permitir que múltiples clientes se conecten al mismo servidor. Todos los transportes deben gestionar todo el ciclo de vida de la conexión, tema que se aborda en [Link to Come].

**Soporte de Múltiples Transportes**
Al construir un cliente o un servidor, no estás limitado a usar un solo transporte. Al soportar múltiples transportes, le das a los usuarios la flexibilidad de elegir el transporte más adecuado para su caso de uso. Anthropic recomienda encarecidamente que siempre se soporte el transporte *stdio*, y se puede añadir soporte para *Streamable HTTP* si se planea desplegar un servidor remoto o permitir que la aplicación añada servidores remotos.

La función principal de los transportes es gestionar conexiones y manejar mensajes. Independientemente del transporte usado, los mensajes enviados entre cliente y servidor son siempre mensajes JSON-RPC. Esto significa que el transporte se encarga de cargar los mensajes en el formato correcto para que el cliente y el servidor los comprendan, algo que los transportes incorporados manejan automáticamente. Hay tres tipos principales de mensajes en MCP: solicitudes (*requests*), respuestas (*responses*) y notificaciones (*notifications*). Las solicitudes pueden ser enviadas por el cliente o el servidor y se utilizan para iniciar una acción. Las respuestas son la contestación a las solicitudes e incluyen el resultado de la operación o información de error si ésta falla. Las notificaciones son mensajes unidireccionales que pueden enviar tanto el cliente como el servidor, normalmente para informar sobre cambios de estado.

Como cualquier parte del protocolo, puedes construir tus propios transportes y usarlos de la misma manera que los incorporados. Esto es menos común, pero puede ser útil si tienes necesidades únicas, especialmente en temas de rendimiento y seguridad. Los transportes deben considerarse un límite de seguridad, y si tus necesidades de seguridad superan lo que ofrecen los transportes incorporados, construir uno propio es una buena opción. También puedes crear tu propio transporte si estás integrando MCP con un sistema existente que ya utiliza un protocolo de comunicación o sistema de mensajería. Más detalles sobre la estructura de los transportes y los mensajes se abordarán en [Link to Come].

**El Ciclo de Vida de la Conexión MCP**
Antes de avanzar a los siguientes capítulos, repasemos brevemente el ciclo de vida de la conexión MCP, que regula cómo cada componente maneja la creación, uso y cierre de una conexión, resumido en tres fases principales:

* **Inicialización:** El cliente y el servidor intercambian mensajes para establecer qué versiones del protocolo son compatibles, compartir y negociar capacidades soportadas y comunicar detalles de implementación relevantes para la conexión.
* **Operación:** Es el “bucle” principal donde ocurre la actividad. Basándose en las capacidades negociadas en la fase de inicialización, el cliente envía solicitudes al servidor, y éste responde con resultados de la operación solicitada o un error si la operación falla.
* **Cierre:** Durante esta fase, el cliente termina la conexión del protocolo, siendo el transporte responsable de comunicar esta terminación al servidor.

Si se usan los transportes incorporados para el cliente o servidor, el ciclo de vida de la conexión está casi totalmente gestionado, por lo que es menos relevante enfocarse en ello. Simplemente se debe conectar y desconectar usando el administrador de sesiones que provee el SDK en el lenguaje elegido. El protocolo también permite autorización en la capa de transporte, y los desarrolladores de transportes que incluyan autorización deben implementar soporte para [OAuth 2.1](https://oauth.net/2.1/). Estos temas se tratarán con más detalle en [Link to Come].


**Utilidades del Protocolo y Otras Herramientas para Desarrolladores**

Parte del rápido ascenso de MCP se debe a las herramientas que lo rodean, tanto las creadas por Anthropic como las desarrolladas por terceros en el ecosistema MCP. Dentro del protocolo existe soporte para mensajes de utilidad especiales que pueden ser usados tanto por clientes como por servidores, mientras que fuera del protocolo se encuentran SDKs, herramientas de prueba y depuración, frameworks y más. Estas herramientas, especialmente los frameworks en etapas tempranas, hicieron que interactuar con el protocolo fuera muy sencillo desde el principio, contribuyendo a su rápida adopción. Las herramientas más recientes se centran en mejorar la experiencia del desarrollador, permitiendo revisiones importantes de los SDK oficiales, pruebas directas de servidores MCP, visualización de logs y tareas similares enfocadas al desarrollo. Esto será crucial para la adopción continua de MCP, más allá de su ya impresionante popularidad, dado que los desarrolladores trabajan sobre y alrededor del protocolo.

**Desarrolladores, desarrolladores, desarrolladores, desarrolladores!**
— Steve Ballmer

Aunque los servidores cuentan con sus propias utilidades especializadas, el protocolo base también soporta utilidades comunes que pueden ser usadas por clientes y servidores, normalmente para obtener retroalimentación o cambiar el estado de la conexión. La primera utilidad es la **cancelación**, que se presenta como un mensaje de notificación del cliente o servidor que contiene un ID de solicitud y un motivo de cancelación, y se usa para cancelar solicitudes que todavía están en proceso. **Ping** está disponible como un mensaje JSON-RPC sin parámetros, enviado por el cliente o el servidor para verificar si la conexión sigue activa. Finalmente, **informes de progreso** están disponibles a nivel de protocolo y permiten que clientes o servidores muestren al usuario una barra de progreso precisa que comunica el avance de cualquier operación. Más detalles sobre estas funcionalidades se abordarán en los capítulos 3 y 4.

**Nota:**
Para usar estas herramientas, es necesario que tanto el cliente como el servidor las soporten.

Para quienes construyen con MCP, ya existen varias herramientas de desarrollo diseñadas para facilitar la vida del programador. Además de SDKs en varios lenguajes (Python, Typescript, Go, Rust, C# y otros), la forma principal de interactuar con MCP como desarrollador es mediante los SDK oficiales. Estos “implementan el protocolo”, usando su lenguaje para gestionar, manejar y enrutar los mensajes que son el núcleo de MCP, y están disponibles en Python, Typescript, Go, Rust, C# y más. Todos manejan la funcionalidad central de clientes, servidores y transportes.

Otra herramienta popular es **MCP Inspector**, que proporciona un cliente web y un proxy para conectarse a tus servidores a través de cualquier transporte incorporado y probarlos visualmente, como se muestra en la Figura 2-5.

**Figura 2-5.** Interfaz de MCP Inspector, mostrando la configuración de conexión en la barra lateral izquierda, los selectores de funciones del servidor en el menú superior y la interfaz de inspección de herramientas en el centro.



<img src="../images/mcp_inspector.png" width="800" height="150">


Figure 2-5. The MCP Inspector UI, showing the connection setup in the left menubar, the server function selectors in the top menu, and the tool inspector UI in the center.


La interfaz de MCP Inspector permite una visibilidad profunda del funcionamiento de un servidor MCP individual. Permite inspeccionar recursos, prompts, herramientas y roots, ejecutar herramientas, enviar mensajes de ping e incluso ejecutar flujos de trabajo de *sampling* y *elicitation*, posibilitando probar manualmente todos los aspectos de un servidor MCP. Más información sobre MCP Inspector se abordará en [Link to Come].

En los capítulos siguientes se explorará cómo funcionan los componentes principales de MCP, cómo puede lucir su código usando el SDK de Python y cómo implementar tus propios clientes y servidores. Este contexto —qué inspiró MCP, los problemas que resuelve, cómo se utiliza hoy y cómo fue diseñado— proporcionará una comprensión más profunda de los componentes que aprenderás a usar a continuación.


- https://www.instructa.ai/ai-prompts?development_process=coding
- https://cursor.directory/rules/python
- https://github.com/PacktPublishing/Modern-AI-Agents/tree/main/chr4
- https://github.com/kylestratis/ai_agents_mcp_examples/blob/main/ch3/02_host_w_client_interface/agent.py